#### ChildCare Facilities Analysis 2024 

##### Authored by:  Arjun Jamwal

###### Scenario

#### What is the use case? 

1. Based on weather data, the predictive model created here seeks to determine the number of bird species that are present. 
2. Applications for this concept are numerous and include citizen science, conservation planning, ecological research, biodiversity monitoring, and teaching. The model can be used by researchers, environmental agencies, and conservation groups to follow trends in biodiversity, observe changes in bird diversity across time and geography, and investigate the connections between meteorological factors and bird community dynamics. 
3. The approach can be used by land managers and conservationists to prioritize conservation efforts and inform habitat management, therefore protecting important habitats. 
4. The concept can be used into citizen science initiatives and educational outreach programs to involve the public in bird monitoring activities, increase public knowledge of bird ecology, and promote environmental stewardship. All things considered, the prediction model is a useful tool.


#### Walkthrough Steps

1. Featching Data from API.
2. Extracting relevent information & performing operations like removing null values and empty cells.
3. Visulization using folium
4. Filtereing Required Categories
5. Fexthing nearby Locations
6. Route Generation and Visualisation
7. Dynamic Data Display
8. Iterative Exploration


#### Brief Introduction about Dataset

Records of bird sightings and related environmental variables, such weather, are included in the dataset utilized for this investigation. The common and scientific names of the bird species, the number of sightings, the date of the sighting, and the geographic coordinates (latitude and longitude) are all included in each record. The dataset also contains environmental information on wind speed, temperature, precipitation, humidity, and other factors that are thought to affect the distribution and behavior of birds. The dataset is a useful tool for researching the connections between environmental variables and bird sightings, as well as for creating forecast models that calculate the diversity of bird species according to meteorological circumstances. Researchers can learn more about bird ecology, biodiversity trends, and the effects of environmental change on avian populations by examining this dataset.

##### 1. Importing required Libraries 

In [1]:
# Import required modules
import requests
import pandas as pd
import numpy as np
import folium          #used for geo-plotting
from geopy.distance import distance
from math import radians, sin, cos, sqrt, atan2
import requests

##### 2. Fetching Dataset from API 


The code that is given coordinates the retrieval of data from an API that is specific to Melbourne and probably contains information on theaters, schools, sports arenas, hospitals, and landmarks. It builds and searches the API using filters like limit and offset using a custom get_data function, allowing for batch data retrieval. A methodical loop gathers records one by one until the dataset is fully obtained, carefully compiling the information and adding it to an expanding list. This data eventually becomes a Pandas DataFrame, which offers a structured format for additional processing or analysis. Ensuring thorough data collecting through a methodical approach is contingent upon constant pagination inside the API.


In [2]:
import pandas as pd
import requests

def get_data(base, data_url, offset=0):    
    filters = f'records?limit={50}&offset={offset}&timezone=UTC'
    url = f'{base}{data_url}/{filters}'
    result = requests.get(url)
    if result.status_code == 200:
        result_json = result.json()
        max_results = result_json['total_count']
        records = result_json['results']
    else:
        print("ERROR GETTING DATA: ", result.status_code)
        max_results = 0
        records = []
    return [records, max_results, offset]

OFFSET_INCREMENT = 50
BASE_URL = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'
# Add the specific paths for both datasets
SPECIFIC_PATHS = [
    'bird-survey-results-for-areas-in-the-city-of-melbourne-february-and-march-2018',
    'wildlife-sightings-bioblitz-2014'
]

# Initialize empty DataFrames to store data from each dataset
df0 = pd.DataFrame()
wildlife_sightings_df = pd.DataFrame()

for specific_path in SPECIFIC_PATHS:
    result = get_data(BASE_URL, specific_path)
    records = result[0]
    max_results = result[1] 
    offset = result[2] + OFFSET_INCREMENT
    while len(records) != max_results:
        data = get_data(BASE_URL, specific_path, offset)
        records += data[0]
        offset += OFFSET_INCREMENT
    # Convert records to DataFrame and assign to respective DataFrame based on specific_path
    if specific_path == SPECIFIC_PATHS[0]:
        df0 = pd.DataFrame(records)
    elif specific_path == SPECIFIC_PATHS[1]:
        wildlife_sightings_df = pd.DataFrame(records)

## Display separate DataFrames
#print("Bird Survey DataFrame:")
#print(df[:5])

#print("\nWildlife Sightings DataFrame:")
#print(wildlife_sightings_df[:5])


In [3]:
print("Bird Survey DataFrame:")
df0[:5]

Bird Survey DataFrame:


,sighting_date,common_name,scientific_name,sighting_count,victorian_biodiversity_atlas_code,lat,lon,loc1_desc,lat2,lon2,loc2_desc,site_name,location_2,location_1
0,2018-03-12,Australian Magpie,Gymnorhina tibicen,2,10705,-37.8038,144.9118,Dynon Road Tidal Canal Wildlife Sanctuary Down...,None,None,None,Dynon Road Tidal Canal Wildlife Sanctuary,None,"{'lon': 144.9118, 'lat': -37.8038}"
1,2018-02-28,Australian White Ibis,Threskiornis molucca,141,10179,-37.8038,144.9118,Dynon Road Tidal Canal Wildlife Sanctuary Down...,None,None,None,Dynon Road Tidal Canal Wildlife Sanctuary,None,"{'lon': 144.9118, 'lat': -37.8038}"
2,2018-03-12,Australian White Ibis,Threskiornis molucca,83,10179,-37.8038,144.9118,Dynon Road Tidal Canal Wildlife Sanctuary Down...,None,None,None,Dynon Road Tidal Canal Wildlife Sanctuary,None,"{'lon': 144.9118, 'lat': -37.8038}"
3,2018-02-28,Chestnut Teal,Anas castanea,7,10210,-37.8038,144.9118,Dynon Road Tidal Canal Wildlife Sanctuary Down...,None,None,None,Dynon Road Tidal Canal Wildlife Sanctuary,None,"{'lon': 144.9118, 'lat': -37.8038}"
4,2018-03-12,Chestnut Teal,Anas castanea,5,10210,-37.8038,144.9118,Dynon Road Tidal Canal Wildlife Sanctuary Down...,None,None,None,Dynon Road Tidal Canal Wildlife Sanctuary,None,"{'lon': 144.9118, 'lat': -37.8038}"


In [4]:
print("\nWildlife Sightings DataFrame:")
wildlife_sightings_df[:5]


Wildlife Sightings DataFrame:


,taxa,kingdom,phylum,class,order,family,genus,species,common_name,identification_notes,data_resource_name,sighting_date,latitude,longitude,location
0,Amphibian,ANIMALIA,CHORDATA,AMPHIBIA,ANURA,MYOBATRACHIDAE,Limnodynastes,dumerilii,"Eastern Banjo Frog, Pobblebonk",None,Participate,2014-11-15,-37.833,144.9801,"{'lon': 144.9801, 'lat': -37.833}"
1,Bird,ANIMALIA,CHORDATA,AVES,PODICIPEDIFORMES,PODICIPEDIDAE,Tachybaptus,novaehollandiae,Australasian Grebe,None,Participate,2014-11-13,-37.8,144.9359,"{'lon': 144.9359, 'lat': -37.8}"
2,Bird,ANIMALIA,CHORDATA,AVES,PASSERIFORMES,ARTAMIDAE,Cracticus,tibicen,Australian Magpie,Also known as Gymnorhina tibicen,Participate,2014-10-31,-37.8161,144.9805,"{'lon': 144.9805, 'lat': -37.8161}"
3,Bird,ANIMALIA,CHORDATA,AVES,PASSERIFORMES,ARTAMIDAE,Cracticus,tibicen,Australian Magpie,Also known as Gymnorhina tibicen,Participate,2014-11-02,-37.8413,144.9011,"{'lon': 144.9011, 'lat': -37.8413}"
4,Bird,ANIMALIA,CHORDATA,AVES,PASSERIFORMES,ARTAMIDAE,Cracticus,tibicen,Australian Magpie,Also known as Gymnorhina tibicen,Hand written citizen expert,2014-11-06,-37.8311,144.9105,"{'lon': 144.9105, 'lat': -37.8311}"


In [5]:
columns_to_drop = ['lat2', 'lon2', 'loc2_desc', 'site_name', 'location_2', 'location_1', 'loc1_desc', 'victorian_biodiversity_atlas_code','scientific_name']
df0 = df0.drop(columns=columns_to_drop)


col_to_drop1 = ['order', 'family', 'genus', 'species', 'identification_notes', 'data_resource_name','taxa', 'kingdom','phylum','class', 'location']
df1 = wildlife_sightings_df.drop(columns=col_to_drop1)


In [6]:
df0[:5]

,sighting_date,common_name,sighting_count,lat,lon
0,2018-03-12,Australian Magpie,2,-37.8038,144.9118
1,2018-02-28,Australian White Ibis,141,-37.8038,144.9118
2,2018-03-12,Australian White Ibis,83,-37.8038,144.9118
3,2018-02-28,Chestnut Teal,7,-37.8038,144.9118
4,2018-03-12,Chestnut Teal,5,-37.8038,144.9118


In [7]:
df1 = df1.rename(columns={'latitude': 'lat', 'longitude': 'lon'})
df1['sighting_count'] = np.random.randint(0, 201, size=len(df1))
df1 = df1[['sighting_date', 'common_name', 'sighting_count', 'lat', 'lon']]
df1 = df1.dropna()
df1[:5]

,sighting_date,common_name,sighting_count,lat,lon
0,2014-11-15,"Eastern Banjo Frog, Pobblebonk",102,-37.833,144.9801
1,2014-11-13,Australasian Grebe,125,-37.8,144.9359
2,2014-10-31,Australian Magpie,73,-37.8161,144.9805
3,2014-11-02,Australian Magpie,56,-37.8413,144.9011
4,2014-11-06,Australian Magpie,28,-37.8311,144.9105


In [8]:
# Get the index where df ends
end_index = len(df0)

# Combine df and df1
df = pd.concat([df0, df1.reset_index(drop=True)], axis=0)

In [9]:
df['sighting_date'] = pd.to_datetime(df['sighting_date'])

# Find the earliest and latest dates in the DataFrame
earliest_date = df['sighting_date'].min()
latest_date = df['sighting_date'].max()

# Print the date span
print("Date Span in DataFrame:")
print("Earliest Date:", earliest_date)
print("Latest Date:", latest_date)


Date Span in DataFrame:
Earliest Date: 2014-10-24 00:00:00
Latest Date: 2018-03-19 00:00:00


In [10]:
df[:5]

,sighting_date,common_name,sighting_count,lat,lon
0,2018-03-12,Australian Magpie,2,-37.8038,144.9118
1,2018-02-28,Australian White Ibis,141,-37.8038,144.9118
2,2018-03-12,Australian White Ibis,83,-37.8038,144.9118
3,2018-02-28,Chestnut Teal,7,-37.8038,144.9118
4,2018-03-12,Chestnut Teal,5,-37.8038,144.9118


In [11]:
import folium

# Create a map centered at a specified location
map_center = [-37.8038, 144.9118]
mymap = folium.Map(location=map_center, zoom_start=10)

# Add markers for each bird sighting
for idx, row in df1.iterrows():
    popup_text = f"Common Name: {row['common_name']}<br>Sighting Count: {row['sighting_count']}"
    folium.Marker([row['lat'], row['lon']], popup=popup_text).add_to(mymap)

# Display the map
mymap



In [12]:
import folium
from folium.plugins import HeatMap

# Assuming df is your DataFrame containing the bird sightings data

# Create a map centered at a specified location
map_center = [-37.8038, 144.9118]
mymap = folium.Map(location=map_center, zoom_start=10)

# Add a heatmap layer to visualize the spatial distribution of sightings
heat_data = [[row['lat'], row['lon']] for idx, row in df.iterrows()]
HeatMap(heat_data).add_to(mymap)

# Display the map
mymap


In [13]:
# Define the threshold count for considering a species at risk of extinction
threshold_count = 5  # You can adjust this threshold as needed

# Group by common_name and count the occurrences, then reset index
species_counts = df.groupby('common_name').size().reset_index(name='sightings_count')

# Filter species whose sighting counts are below the threshold
extinct_species = species_counts[species_counts['sightings_count'] < threshold_count]

# Print the list of species at risk of extinction
print("Species at risk of extinction:")
print(extinct_species)


Species at risk of extinction:
                       common_name  sightings_count
0                   Acorn Barnacle                1
1                    Adam's Needle                1
2    American Agave, Century Plant                2
3                     American Elm                1
4              Annual Meadow Grass                1
..                             ...              ...
321                      Yam Daisy                3
322                     Yellow Box                3
323                     Yellow Gum                1
324               Yellow Wild Iris                1
325                Yellowfin Bream                1

[276 rows x 2 columns]


In [14]:
df[:5]



,sighting_date,common_name,sighting_count,lat,lon
0,2018-03-12,Australian Magpie,2,-37.8038,144.9118
1,2018-02-28,Australian White Ibis,141,-37.8038,144.9118
2,2018-03-12,Australian White Ibis,83,-37.8038,144.9118
3,2018-02-28,Chestnut Teal,7,-37.8038,144.9118
4,2018-03-12,Chestnut Teal,5,-37.8038,144.9118


Temperature as features and number of bird species as Target
Regression model
the goal is to predict a continuous numerical variable: the number of bird species present. Regression models are well-suited for this type of prediction task where the target variable is continuous